## SSL Enabled Requests

Install seldon with ingress provider.


In [ ]:
%%bash
kubectl create ns cert-manager
helm install \
  cert-manager jetstack/cert-manager \
  --namespace cert-manager \
  --version v0.15.1 \
  --set installCRDs=true

#### Deploying self signed certificate

In [8]:
%%bash
kubectl apply -f - << END
apiVersion: cert-manager.io/v1alpha2
kind: Issuer
metadata:
  name: selfsigned-issuer
  namespace: default
spec:
  selfSigned: {}
END

issuer.cert-manager.io/selfsigned-issuer created


In [4]:
!kubectl get issuer 

NAME                READY   AGE
selfsigned-issuer   True    23s


In [9]:
%%bash
kubectl apply -f - << END
apiVersion: cert-manager.io/v1alpha2
kind: Certificate
metadata:
  name: selfsigned-crt
spec:
  secretName: selfsigned-crt-secret
  dnsNames:
  - foo.example.com
  issuerRef:
    kind: Issuer
    group: cert-manager.io
    name: selfsigned-issuer
END

certificate.cert-manager.io/selfsigned-crt created


In [11]:
!kubectl get certificate -o yaml

apiVersion: v1
items:
- apiVersion: cert-manager.io/v1alpha3
  kind: Certificate
  metadata:
    annotations:
      kubectl.kubernetes.io/last-applied-configuration: |
        {"apiVersion":"cert-manager.io/v1alpha2","kind":"Certificate","metadata":{"annotations":{},"name":"selfsigned-crt","namespace":"default"},"spec":{"dnsNames":["foo.example.com"],"issuerRef":{"group":"cert-manager.io","kind":"Issuer","name":"selfsigned-issuer"},"secretName":"selfsigned-crt-secret"}}
    creationTimestamp: "2020-06-19T18:35:01Z"
    generation: 1
    name: selfsigned-crt
    namespace: default
    resourceVersion: "630874"
    selfLink: /apis/cert-manager.io/v1alpha3/namespaces/default/certificates/selfsigned-crt
    uid: f7196e82-6bc8-42fb-899e-8f53a408bdce
  spec:
    dnsNames:
    - foo.example.com
    issuerRef:
      group: cert-manager.io
      kind: Issuer
      name: selfsigned-issuer
    secretName: selfsigned-crt-secret
  status:
    conditions:
    - lastTransitionTime: "2020-06-19T18:35: